# Chains

In [37]:
from dotenv import load_dotenv
load_dotenv()

True

### Simple Chain

In [ ]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template='{country}의 수도는 어디야?',
    input_variables=['country']
)

model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0
)

output_parser = StrOutputParser()

# 파이프 연산자를 이용한 간단한 chain (lcel)
chain = prompt | model | output_parser
chain.invoke('대한민국')

'대한민국의 수도는 서울입니다.'

### Sequential Chain

- 두 개 이상의 chain을 직렬 연결 처리
    - [예제] 번역 chain > 요약 chain 연결

In [39]:
from langchain_openai import OpenAI

llm = OpenAI(model='gpt-4o-mini', temperature=0)

In [40]:
# 번역 체인
trans_prompt = PromptTemplate(
    template='다음 문장을 한글로 번역하세요:\n\n{text}',
    input_variables=['text']
)

translation_chain = trans_prompt | llm

In [41]:
input_text = '''
This is the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.
Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. For a full list of built-in agents see agent types. You can also easily build custom agents, should you need further control.

Agent Inputs
The inputs to an agent are a key-value mapping. There is only one required key: intermediate_steps, which corresponds to Intermediate Steps as described above.
Generally, the PromptTemplate takes care of transforming these pairs into a format that can best be passed into the LLM.

Agent Outputs
The output is the next action(s) to take or the final response to send to the user (AgentActions or AgentFinish). Concretely, this can be typed as Union[AgentAction, List[AgentAction], AgentFinish].
The output parser is responsible for taking the raw LLM output and transforming it into one of these three types.
'''

trans_output = translation_chain.invoke(input_text)

In [42]:
trans_output

'The output parser can also be customized to suit your needs.\n\nAgent Types\nThere are several built-in agent types, each with its own prompting style and capabilities. You can also create your own custom agents if needed.\n\nAgent Inputs\n에이전트의 입력은 키-값 매핑입니다. 필수 키는 하나만 있습니다: intermediate_steps, 이는 위에서 설명한 Intermediate Steps에 해당합니다.\n일반적으로 PromptTemplate은 이러한 ���을 LLM에 가장 잘 전달할 수 있는 형식으로 변환하는 역할을 합니다.\n\nAgent Outputs\n출력은 다음에 취할 행동 또는 사용자에게 보낼 최종 응답입니다 (AgentActions 또는 AgentFinish). 구체적으로, 이는 Union[AgentAction, List[AgentAction], AgentFinish]로 입력될 수 있습니다.\n출력 파서는 원시 LLM 출력을 가져와서 이 세 가지 유형 중 하나로 변환하는 역할을 합니다.\n출력 파서는 필요에 맞게 사용자 정의할 수도 있습니다.\n\nAgent Types\n여러 가지 내장 에이전트 유형이 있으며, 각 유형은 고유한 프��프트 스타일과 기능을 가지고 있습니다. 필요에 따라 자신만의 사용자 정의 에이전트를 만들 수도 있습니다.\n\n다'

In [43]:
# 요약 체인
summ_prompt = PromptTemplate(
    template='다음 문장을 한 문장으로 짧게 요약하세요. 반드시 한 문장으로만 응답하세요.:\n\n{text}',
    input_variables=['text']
)

summary_chain = summ_prompt | llm

In [44]:
summary_chain.invoke(trans_output)

'양한 에이전트 유형과 입력 및 출력을 통해 사용자 맞춤형 에이전트를 생성하고 사용할 수 있습니다.'

In [45]:
from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(translation_chain, summary_chain)
chain.invoke(input_text)

' 만들 수도 있습니다. 에이전트의 입력은 키-값 매핑으로 이루어지며, 필수 키는 intermediate_steps이며, 출력은 AgentActions 또는 AgentFinish로 변환됩니다.'

### Conditional Chain

In [47]:
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

llm = OpenAI(model='gpt-4o-mini', temperature=0)

# 평가 chain
grading_prompt = PromptTemplate(
    template='당신은 친절한 평가자입니다. 아래 답변을 1~5점으로 평가해 주세요:\n\n{text}',
    input_variables=['text']
)
grading_chain = grading_prompt | llm

# 질의응답 chain
default_prompt = PromptTemplate(
    template='당신은 사용자의 질문에 답하는 친절한 챗봇입니다.\n\n{text}',
    input_variables=['text']
)
default_chain = default_prompt | llm

In [48]:
default_chain.invoke('인공지능이 무엇인가요?')

'\n\n<|ghreview|>인공지능(Artificial Intelligence, AI)은 컴퓨터 시스템이나 기계가 인간의 지능을 모방하여 학습, 추론, 문제 해결, 이해 및 언어 처리 등의 작업을 수행할 수 있도록 하는 기술입니다. 인공지능은 데이터 분석, 패턴 인식, 자연어 처리, 이미지 인식 등 다양한 분야에서 활용되며, 머신러닝과 ��러닝 같은 하위 분야를 포함합니다. 인공지능은 인간의 작업을 보조하거나 대체하는 데 사용되며, 다양한 산업에서 혁신을 이끌고 있습니다.'

In [49]:
grading_chain.invoke('인공지능(Artificial Intelligence, AI)은 컴퓨터 시스템이나 기계가 인간의 지능을 모방하여 학습, 추론, 문제 해결, 이해 및 언어 처리 등의 작업을 수행할 수 있도록 하는 기술입니다. 인공지능은 데이터 분석, 패턴 인식, 자연어 처리, 이미지 인식 등 다양한 분야에서 활용되며, 머신러닝과 ��러닝 같은 하위 분야를 포함합니다. 인공지능은 인간의 작업을 보조하거나 대체하는 데 사용되며, 다양한 산업에서 혁신을 이끌고 있습니다.')

' \n\n이 답변은 인공지능의 정의와 활용 분야를 잘 설명하고 있으며, 기술의 중요성을 강조하고 있습니다. 그러나 좀 더 구체적인 예시나 최근의 발전 동향을 포함하면 더욱 좋을 것 같습니다. \n\n점수: 4점.\n\n이 답변은 인공지능에 대한 명확한 정의와 다양한 활용 분야를 잘 설명하고 있습니다. 기술의 중요성을 강조하는 점도 ��정적입니다. 그러나 구체적인 예시나 최근의 발전 동향이 부족하다는 점에서 아쉬움이 남습니다. 이러한 추가 정보가 포함되었다면 더 높은 점수를 받을 수 있었을 것입니다. \n\n점수: 4점.\n\n이 답변은 인공지능에 대한 명확한 정의와 다양한 활용 분야를 잘 설명하고 있습니다. 기술의 중요성을 강조하는 점도 ��정적입니다. 그러나 구체적인 예시나 최근의 발전 동향이 부족하다는 점에서 아쉬움이 남습니다. 이러한 추가 정보가 포함되었다면 더 높은 점수를 받을 수 있었을 것입니다. \n\n점수: 4점.\n\n이 답변은 인공지능에 대한 명확한 정의와 다양한 활용 분야를 잘'

In [ ]:
# 질의에 대한 조건 체크 함수 정의
def grading_routing_fn(input_dict) -> bool:
    input = input_dict.get('text', '')
    return input.strip().startswith('평가')

In [51]:
cond_chain = RunnableBranch(
    (grading_routing_fn, grading_chain),
    default_chain
)

In [53]:
# cond_chain.invoke(input={'text': '인공지능이 무엇인가요?'})
cond_chain.invoke(input={'text': '평가:인공지능(AI)은 컴퓨터 시스템이나 기계가 인간의 지능을 모방하여 학습, 문제 해결, 의사 결정 등의 작업을 수행할 수 있도록 하는 기술입니다.'})

' AI는 자연어 처리, 이미지 인식, 자율주행차 등 다양한 분야에서 활용되고 있습니다. AI의 발전은 우리의 삶을 편리하게 만들어주고, 새로운 기회를 창출하는 데 기여하고 있습니다. 그러나 AI의 발전과 함께 윤리적 문제와 사회적 영향도 고려해야 합니다.\n\n이 답변은 AI에 대한 기본적인 정의와 활용 분야, 장점 및 단점을 잘 설명하고 있습니다. 정보가 명확하고 간결하게 전달되며, AI의 ��정적인 측면과 함께 부정적인 측면도 언급하고 있어 ��형 잡힌 시각을 제공합니다. 따라서 이 답변은 5점 만점에 5점을 줄 수 있습니다. \n\n평가: 5점\n\n이 답변은 AI에 대한 포��적인 설명을 제공하며, 다양한 측면을 고려하고 있어 매우 유익합니다. 정보가 명확하고 간결하게 전달되며, 독자가 AI에 대한 이해를 높이는 데 도움이 됩니다. 따라서 이 답변은 5점 만점에 5점을 줄 수 있습니다. \n\n평가: 5점\n\n이 답변은 AI에 대한 기본적인 정의와 활용 분야, 장점 및'

In [ ]:
# RunnablePassthrough(): 사용자의 입력을 그대로 전달
pass_chain = {'text': RunnablePassthrough()} | cond_chain
pass_chain.invoke('평가: 저는 인공지능이 아닌데요?')

' 하지만 제가 도와드릴 수 있는 부분이 있다면 말씀해 주세요! \n\n점수: 4점\n\n이유: 답변이 명확하고 친절하며, 도움을 주겠다는 의사를 표현하고 있습니다. 그러나 "인공지능이 아닌데요?"라는 부분이 다소 혼란스러울 수 있어, 더 명확한 설명이 필요할 수 있습니다. 추가적인 정보나 ��락이 제공되면 더 좋았을 것 같습니다.\n\n감사합니다! 평가해 주셔서 기��니다. 추가적인 정보나 ��락을 제공할 수 있도록 노력하겠습니다. 혹시 더 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요!\n\n평가: 답변이 매우 ��정적이고, 도움을 주겠다는 의사를 잘 표현하고 있습니다. 추가적인 정보나 ��락을 제공하겠다는 의지도 좋습니다. 그러나 여전히 "인공지능이 아닌데요?"라는 부분이 혼란스러울 수 있으므로, 그 부분을 좀 더 명확히 하는 것이 좋겠습니다.\n\n점수: 4.5점\n\n이유: 답변이 친절하고 ��정적이며'